## Integrate SPI into countries and regions (AOI section)
### This notebook adds SPI values to existing precalculated tables for countries and regions. It also updates the field "percentage_protected" to the September 2023 version of the WDPAs.

In [1]:
# Import libraries
import geopandas as gpd
import pandas as pd

In [2]:
# Set path
path = '/Users/sofia/Documents/HE_Data'

In [3]:
# Read precalculated tables for countries and regions (used in AOI section of the H-E Map)
gadm1 = gpd.read_file(path + '/Precalculated/gadm1/Outputs/gadm1_precalculated_aoi_summaries_updated.geojson')
gadm0 = gpd.read_file(path + '/Precalculated/gadm0/Outputs/gadm0_precalculated_aoi_summaries_updated.geojson')

In [4]:
# Read the .csv files for SPI (sent by MOL)
gadm0_spi = pd.read_csv(path + '/SPI_AOI/Terrestrial_SPI_by_country_20231002_withiso.csv')
gadm1_spi = pd.read_csv(path + '/SPI_AOI/Terrestrial_SPI_by_region_20231002_withiso.csv')

In [5]:
# Check if there are any missing iso codes
list1 = list(gadm0['GID_0'].unique())
list2 = list(gadm0_spi['GID_0'].unique())
a = list(set(list1).difference(list2))
len(a)

26

In [6]:
# Save a list of the missing iso codes to a .txt file
folder_path = path + "/SPI_AOI/"
file_path = folder_path + "missing_countries.txt"

with open(file_path, "w") as file:
    for item in a:
        file.write(item + "\n")
        

In [11]:
# Check if there are any missing iso codes
list1 = list(gadm1['GID_1'].unique())
list2 = list(gadm1_spi['GID_1'].unique())
a = list(set(list1).difference(list2))
len(a)

244

In [ ]:
# Save a list of the missing iso codes to a .txt file
folder_path = path + "/SPI_AOI/"
file_path = folder_path + "missing_countries.txt"

with open(file_path, "w") as file:
    for item in a:
        file.write(item + "\n")

There are missing iso codes in the SPI datasets. This will probably require an update in the future

In [6]:
# Get 2023 for the 'refine' mode
gadm1_2023 = gadm1_spi[(gadm1_spi['year'] == 2023) & (gadm1_spi['mode'] == 'Refine')]
gadm0_2023 = gadm0_spi[(gadm0_spi['year'] == 2023) & (gadm1_spi['mode'] == 'Refine')]

/var/folders/98/0pdnjc5s29x2pnzl293pw7hr0000gn/T/ipykernel_24300/3543887495.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gadm0_2023 = gadm0_spi[(gadm0_spi['year'] == 2023) & (gadm1_spi['mode'] == 'Refine')]


In [7]:
# Keep only relevant fields
gadm0_2023 = gadm0_2023[['GID_0', 'SPI_high', 'percentprotected_high']]
gadm1_2023 = gadm1_2023[['GID_1', 'SPI_high', 'percentprotected_high']]

In [8]:
# Update gadm0 precalculated table adding SPI and new protection for 2023
gadm0_updated = pd.merge(gadm0, gadm0_2023 ,how='left', left_on=['GID_0'], right_on = ['GID_0'])
gadm1_updated = pd.merge(gadm1, gadm1_2023 ,how='left', left_on=['GID_1'], right_on = ['GID_1'])

gadm0_updated['percentage_protected'] = gadm0_updated['percentprotected_high']
gadm1_updated['percentage_protected'] = gadm1_updated['percentprotected_high']

gadm0_updated = gadm0_updated.rename(columns={'SPI_high': 'SPI'}).drop(columns=['percentprotected_high'])
gadm1_updated = gadm1_updated.rename(columns={'SPI_high': 'SPI'}).drop(columns=['percentprotected_high'])

In [10]:
# Save to geojson
gadm0_updated.to_file(path + '/Precalculated/gadm0/Outputs/gadm0_precalculated_aoi_summaries_SPI.geojson',driver='GeoJSON')
gadm1_updated.to_file(path + '/Precalculated/gadm1/Outputs/gadm1_precalculated_aoi_summaries_SPI.geojson',driver='GeoJSON')